In [2]:
import pandas as pd
import numpy as np
import sqlite3

engine = sqlite3.connect('yahoo')

In [4]:
click_cols = ['time', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6', 'displayed', 'clicked']

In [5]:
articles_df = pd.read_sql_query('SELECT * FROM articles',con=engine).set_index('index')
clicks_df = pd.read_sql_query('''SELECT * FROM clicks WHERE (ABS(CAST((RANDOM()) as int)) % 100) < 5''', con=engine)
clicks_df[click_cols] = clicks_df[click_cols].apply(pd.to_numeric, ) # Temporary workaround, this should be done before sql

In [130]:
#articles_df.dtypes

In [6]:
articles_df

,2,3,4,5,6,1
index,,,,,,
109498,0.306008,0.000450,0.077048,0.230439,0.386055,1.0
109509,0.306008,0.000450,0.077048,0.230439,0.386055,1.0
109508,0.264355,0.000012,0.037393,0.420649,0.277591,1.0
109473,0.295442,0.000014,0.135191,0.292304,0.277050,1.0
109503,0.306008,0.000450,0.077048,0.230439,0.386055,1.0
109502,0.277121,0.000131,0.038153,0.335835,0.348760,1.0
109501,0.249086,0.001009,0.514682,0.067732,0.167491,1.0
109492,0.331830,0.000022,0.019904,0.440390,0.207855,1.0
109495,0.313277,0.000125,0.018413,0.410555,0.257630,1.0


In [118]:
clicks_df.shape

(234140, 11)

In [121]:
#clicks_df.dtypes

In [120]:
 clicks_df.head()

,index,time,user_1,user_2,user_3,user_4,user_5,user_6,article_pool,displayed,clicked
0,24,1241160900,0.008325,0.079014,0.001413,0.911076,0.000172,1.0,"['109498', '109509', '109508', '109473', '1095...",109510,0
1,59,1241160900,0.003677,0.000000,0.986935,0.008073,0.001315,1.0,"['109498', '109509', '109508', '109473', '1095...",109503,0
2,65,1241160900,0.042438,0.150965,0.005879,0.800057,0.000661,1.0,"['109498', '109509', '109508', '109473', '1095...",109495,0
3,85,1241160900,0.553029,0.045877,0.094129,0.291508,0.015457,1.0,"['109498', '109509', '109508', '109473', '1095...",109498,0
4,114,1241160900,0.553029,0.045877,0.094129,0.291508,0.015457,1.0,"['109498', '109509', '109508', '109473', '1095...",109506,0


In [1]:
"""
Evaluation Protocol
===================
Let σ be an input stream of logged events to be used for the simulation,
where an event consists of a tuple consisting of the user vector, 
20 article vectors (context vectors), the displayed vector (selected arm), 
and the result (observed reward).
Step through each element s sampled from the stream.
Let t denote the current time-step in the algorithm and h_{t-1} prior history.
If given h_{t-1} and the current context, the algorithm picks the same vector
as the selected arm - retain the event and add it to the history. Otherwise,
keep processing items.
"""

In [5]:
list(map(lambda x : 1 if x == '109510' else 0, eval(clicks_df.ix[0]['article_pool'])))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [68]:
""" Playing around with how to convert Yahoo data to good format for evaluation. """

from numpy import transpose
from numpy.linalg import inv,pinv, det
from scipy.stats import norm

np.set_printoptions(precision=3)

user_feat_mat = clicks_df.as_matrix(['user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6'])
disp_mat = clicks_df.as_matrix(['displayed']).flatten()

k = 20
d = 12
X = np.empty((0, k, 12))
Y = np.empty((0,20))
T = 2
_delta = 0.1

for t in range(init_iter):
    # Extract all article vectors for first log item.
    _article_pool = eval(clicks_df.ix[t]['article_pool'])
    article_vecs = np.array([articles_df.ix[x].values for x in _article_pool])
    # For each of the article row vectors, add a copy of the user features.
    tmp = np.append(article_vecs, values=np.tile(user_feat_mat[t], (len(article_vecs), 1)), axis=1)
    tmp = tmp.reshape((1, k, d)) # (timestep, arm, feature)
#     X = np.append(X, tmp, axis=0)  # Feature vector of 6 article features and 6 user features
    X = np.vstack([X, tmp])
    _rewards = np.array(list(map(lambda x : 1 if int(x) == disp_mat[t] else 0, _article_pool)))
#     print(_rewards)
    Y = np.vstack([Y, _rewards])
print(X.shape)


i = 8
_Xti = X[:T,i]
# print(_Xti)
_XtiT = transpose(_Xti)
_XTX = pinv(_XtiT.dot(_Xti))
_Yti = Y[:T,i]
Bh_t_i = _XTX.dot(_XtiT).dot(_Yti)
yh_t_i = Bh_t_i.dot(X[T-1,i])
_s2 = np.var(Y[:T,i])
print(Y[:T, i])
w_t_i = norm.ppf(1 - _delta/(2*1000*k), loc=0, 
                 scale=np.sqrt(_s2 * X[T-1,i].dot(_XTX).dot(transpose(X[T-1,i]))))
# print(Y.shape)
w_t_i
print([yh_t_i - w_t_i, yh_t_i + w_t_i])

(2, 20, 12)
[ 1.  0.]
[-2.2823938651397553, 2.2823938651397553]


In [ ]:
def top_interval(X, Y, k, d, _delta, T):
    """
    Simulates T rounds of TopInterval for k.
    
    :param X: a 3-axis (T, k, d) ndarray of d-dimensional context vectors for each
              time-step and arm
    :param Y: a T x k ndarray of reward function output for each context vector
    :param k: the number of arms
    :param d: the number of features
    :param _delta: confidence parameter
    :param T: the number of iterations
    """
    _eta = eta(T)                               # exploration cutoff probabilities
    picks = []
    for t in range(T):
        print('Iteration [{0} / {1}]'.format(t, T))